In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api.formatters import TextFormatter
import requests
import re
import os

def extrair_id_video(url):
    """Extrai o ID do vídeo de URLs comuns do YouTube."""
    # Tenta encontrar v=ID
    match = re.search(r'(?:v=|\/)([0-9A-Za-z_-]{11}).*', url)
    if match:
        return match.group(1)
    return None

def obter_titulo_video(url):
    """Pega o título da página HTML do vídeo para não precisar de libs pesadas."""
    try:
        response = requests.get(url)
        # Busca o conteúdo dentro da tag <title>
        matches = re.findall(r'<title>(.*?)</title>', response.text)
        if matches:
            return matches[0].replace(" - YouTube", "")
    except:
        pass
    return "transcricao_youtube"

def limpar_nome_arquivo(titulo):
    """Remove caracteres inválidos para nome de arquivo."""
    return re.sub(r'[\\/*?:"<>|]', "", titulo)

def salvar_markdown(url, titulo, texto, idioma_detectado=None):
    """Salva o conteúdo em um arquivo .md."""
    nome_arquivo = f"{limpar_nome_arquivo(titulo)}.md"
    
    info_idioma = f"\n**Idioma detectado:** {idioma_detectado}\n" if idioma_detectado else ""
    
    conteudo = f"""# {titulo}

**Link Original:** {url}
{info_idioma}
---

## Transcrição

{texto}
"""
    
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        f.write(conteudo)
    
    print(f"✅ Arquivo salvo: {nome_arquivo}")

def verificar_idiomas_disponiveis(video_id, yt_api):
    """Lista os idiomas disponíveis para o vídeo."""
    try:
        transcript_list = yt_api.list(video_id)
        
        idiomas = []
        for transcript in transcript_list:
            idioma_info = {
                'codigo': transcript.language_code,
                'nome': transcript.language,
                'gerado': transcript.is_generated,
                'translatable': transcript.is_translatable
            }
            idiomas.append(idioma_info)
        
        return idiomas
    except Exception as e:
        return []

def processar_video(url, yt_api, mostrar_detalhes=True):
    """Processa um único vídeo e retorna True se sucesso, False caso contrário."""
    video_id = extrair_id_video(url)
    
    if not video_id:
        if mostrar_detalhes:
            print("  ❌ ID do vídeo não encontrado. Verifique o link.")
        return False

    try:
        # Verifica os idiomas disponíveis
        if mostrar_detalhes:
            print("  🔍 Verificando idiomas disponíveis...")
        idiomas_disponiveis = verificar_idiomas_disponiveis(video_id, yt_api)
        
        if idiomas_disponiveis and mostrar_detalhes:
            # Tenta encontrar o idioma principal (não gerado, se possível)
            idioma_principal = None
            for idioma in idiomas_disponiveis:
                if not idioma['gerado']:
                    idioma_principal = idioma
                    break
            
            # Se não encontrar manual, usa o primeiro disponível
            if not idioma_principal:
                idioma_principal = idiomas_disponiveis[0]
            
            print(f"  🌐 Usando idioma: {idioma_principal['nome']} ({idioma_principal['codigo']})")
        
        # Tenta buscar legendas com prioridade: pt, pt-BR, en, ou qualquer disponível
        idiomas_prioridade = ['pt', 'pt-BR', 'en']
        
        if idiomas_disponiveis:
            # Se temos lista de idiomas, usa os códigos disponíveis
            codigos_disponiveis = [i['codigo'] for i in idiomas_disponiveis]
            # Ordena pela prioridade, mas mantém apenas os disponíveis
            idiomas_prioridade = [lang for lang in idiomas_prioridade if lang in codigos_disponiveis]
            # Adiciona outros idiomas disponíveis no final
            for codigo in codigos_disponiveis:
                if codigo not in idiomas_prioridade:
                    idiomas_prioridade.append(codigo)
        
        # Busca a transcrição
        fetched_transcript = yt_api.fetch(video_id, languages=idiomas_prioridade)
        
        # Obtém o idioma usado
        idioma_usado = fetched_transcript.language
        
        # O formatter converte a lista de tempos em um texto corrido
        formatter = TextFormatter()
        texto_formatado = formatter.format_transcript(fetched_transcript)
        
        titulo = obter_titulo_video(url)
        salvar_markdown(url, titulo, texto_formatado, idioma_usado)
        
        if mostrar_detalhes and idioma_usado:
            print(f"  ✅ Transcrição obtida com sucesso! Idioma: {idioma_usado}")
        
        return True

    except Exception as e:
        if mostrar_detalhes:
            print(f"  ❌ Erro: Não foi possível obter a legenda.")
            print(f"  Detalhe técnico: {e}")
        return False

def ler_links_arquivo(nome_arquivo):
    """Lê links de um arquivo de texto (um por linha) ou CSV."""
    urls = []
    
    try:
        with open(nome_arquivo, 'r', encoding='utf-8') as f:
            primeira_linha = f.readline().strip().lower()
            f.seek(0)  # Volta ao início
            
            # Verifica se é CSV (tem cabeçalho com "URL" ou "url")
            if 'url' in primeira_linha or 'titulo' in primeira_linha:
                # É CSV - pula o cabeçalho e extrai a coluna URL
                import csv
                reader = csv.DictReader(f)
                for row in reader:
                    # Tenta encontrar a coluna URL (case insensitive)
                    url = None
                    for key in row.keys():
                        if 'url' in key.lower():
                            url = row[key].strip()
                            break
                    if url:
                        urls.append(url)
            else:
                # É arquivo de texto simples - uma URL por linha
                for linha in f:
                    url = linha.strip()
                    if url and (url.startswith('http://') or url.startswith('https://')):
                        urls.append(url)
        
        return urls
    except FileNotFoundError:
        print(f"❌ Arquivo '{nome_arquivo}' não encontrado.")
        return []
    except Exception as e:
        print(f"❌ Erro ao ler arquivo: {e}")
        return []

def main():
    # Lê automaticamente do arquivo list.txt
    nome_arquivo = "list.txt"
    
    print(f"\n📂 Lendo links do arquivo: {nome_arquivo}")
    urls = ler_links_arquivo(nome_arquivo)
    
    if not urls:
        print("❌ Nenhum link válido encontrado no arquivo.")
        return
    
    print(f"✅ Encontrados {len(urls)} links para processar.\n")
    
    # Cria uma instância da API (reutilizada para todos os vídeos)
    yt_api = YouTubeTranscriptApi()
    
    # Contadores
    sucessos = 0
    falhas = 0
    
    # Processa cada vídeo sequencialmente
    for idx, url in enumerate(urls, 1):
        print(f"\n{'='*60}")
        print(f"📹 Processando vídeo {idx}/{len(urls)}")
        print(f"🔗 {url}")
        print(f"{'='*60}")
        
        if processar_video(url, yt_api, mostrar_detalhes=True):
            sucessos += 1
        else:
            falhas += 1
    
    # Resumo final
    print(f"\n{'='*60}")
    print("📊 RESUMO FINAL")
    print(f"{'='*60}")
    print(f"✅ Sucessos: {sucessos}")
    print(f"❌ Falhas: {falhas}")
    print(f"📁 Total processado: {len(urls)}")
    print(f"{'='*60}\n")

if __name__ == "__main__":
    main()


📂 Lendo links do arquivo: list.txt
✅ Encontrados 31 links para processar.


📹 Processando vídeo 1/31
🔗 https://www.youtube.com/watch?v=z-P7rqio4Kg&list=PLMtKfZwPN_B35-S6JEV4nq2JKw1fTtEU6&index=2
  🔍 Verificando idiomas disponíveis...
  🌐 Usando idioma: Portuguese (auto-generated) (pt)
✅ Arquivo salvo: Aula 1.2 Dicas de Aprendizagem (Reaprendizagem Criativa).md
  ✅ Transcrição obtida com sucesso! Idioma: Portuguese (auto-generated)

📹 Processando vídeo 2/31
🔗 https://www.youtube.com/watch?v=Op1dlcP1L3M&list=PLMtKfZwPN_B35-S6JEV4nq2JKw1fTtEU6&index=3
  🔍 Verificando idiomas disponíveis...
  🌐 Usando idioma: Portuguese (auto-generated) (pt)
✅ Arquivo salvo: Aula 2.1 Habilidades do Futuro - As 4 Forças (Reaprendizagem Criativa).md
  ✅ Transcrição obtida com sucesso! Idioma: Portuguese (auto-generated)

📹 Processando vídeo 3/31
🔗 https://www.youtube.com/watch?v=na9PHsRxszY&list=PLMtKfZwPN_B35-S6JEV4nq2JKw1fTtEU6&index=4
  🔍 Verificando idiomas disponíveis...
  🌐 Usando idioma: Portuguese (